In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plt.style.use("ggplot")

# Items 2024

In [4]:
df = pd.read_csv("./DATA/Items2024.csv", sep=";")

df.head()

,CodigoItem,DescripcionItem,PrecioItem,CantidadItem,Municipio,Latitud (Y),Longitud (X),Fecha de Ejecución
0,1.1,Canalización tubería de 25 mm en asfalto (FVA),109834.0,1.0,BELLO,6.351080,-75.518336,29.08.2024 0:00
1,1.1,Canalización tubería de 25 mm en asfalto (FVA),109834.0,10.0,ITUANGO,7.064548,-75.683927,06.12.2024 0:00
2,1.1.13,Canalización tubería de 25 mm en andén,55097.0,2.0,CAREPA,7.847447,-76.740111,12.08.2024 0:00
3,1.1.13,Canalización tubería de 25 mm en andén,55097.0,13.6,TURBO,7.930575,-76.623300,27.08.2024 0:00
4,1.1.13,Canalización tubería de 25 mm en andén,55097.0,25.8,CAREPA,7.759252,-76.652718,18.09.2024 0:00


In [5]:
newDF = {
    "CodigoItem": list(),
    "Item": list(),
    "Cantidad": list(),
    "Municipio": list(),
    "Y": list(),
    "X": list(),
    "FechaEjec": list()
}

for r in range(0, len(df), 1):
    if not df["DescripcionItem"][r].split(" ")[0] in ["S.T.I.", "STI", "Accesorios"]:
        for c in list(df):
            newDF["CodigoItem"].append(df["CodigoItem"][r])
            newDF["Item"].append(df["DescripcionItem"][r].replace(" (FVA)", ""))
            newDF["Cantidad"].append(df["CantidadItem"][r])
            newDF["Municipio"].append(df["Municipio"][r])
            newDF["Y"].append(df["Latitud (Y)"][r])
            newDF["X"].append(df["Longitud (X)"][r])
            newDF["FechaEjec"].append(df["Fecha de Ejecución"][r])

pd.DataFrame(newDF).to_csv("./DATA/ItemsEjec.csv", index=False)